BT Career Challenge

1. Check the data, adjust data, delete null value

1.1 Import needed libraries and data

In [2]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

In [3]:
# use pandas import data
df_customer = pd.read_csv('./Data/Data Set.csv')
df_customer

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Monthtomonth,Yes,Electronic check,29.85,29.85,No
1,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Monthtomonth,Yes,Mailed check,53.85,108.15,Yes
2,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Monthtomonth,Yes,Electronic check,70.70,151.65,Yes
3,9305-CDSKC,Female,0,No,No,8,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,Monthtomonth,Yes,Electronic check,99.65,820.5,Yes
4,1452-KIOVK,Male,0,No,Yes,22,Yes,Yes,Fiber optic,No,...,No,No,Yes,No,Monthtomonth,Yes,Credit card (automatic),89.10,1949.4,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,4807-IZYOZ,Female,0,No,No,51,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Bank transfer (automatic),20.65,1020.75,No
7039,9710-NJERN,Female,0,No,No,39,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.15,826,No
7040,9281-CEDRU,Female,0,Yes,No,68,Yes,No,DSL,No,...,No,Yes,Yes,No,Two year,No,Bank transfer (automatic),64.10,4326.25,No
7041,2569-WGERO,Female,0,No,No,72,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,Yes,Bank transfer (automatic),21.15,1419.4,No


In [4]:
# check the datatype and the number of records
df_customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [5]:
# check the number of churn
df_customer.Churn.value_counts()

No     5174
Yes    1869
Name: Churn, dtype: int64

From the inital exploration, we can see we have 21 columns covering customers' features, and the most important feature churn has 1869 customers who left BT services (indicating it is an imbalanced dataset). It is time to validate data including:
- delete unuseful columns, like customerID
- delete null values
- check if customers is unique
- convert some features from string to numeric values

1.2 Sort out data

In [6]:
# drop customerID
df_customer.drop('customerID', axis = 'columns', inplace = True)
df_customer.sample(5)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
1393,Female,0,No,No,7,Yes,No,Fiber optic,No,No,Yes,Yes,Yes,Yes,Monthtomonth,Yes,Electronic check,100.40,715,No
2515,Male,0,No,No,1,Yes,No,DSL,No,No,No,Yes,No,No,Monthtomonth,No,Credit card (automatic),49.00,49,No
1965,Female,0,Yes,Yes,28,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Monthtomonth,Yes,Bank transfer (automatic),19.55,543.8,No
3562,Male,1,No,No,50,Yes,Yes,DSL,Yes,Yes,No,No,Yes,No,Monthtomonth,Yes,Bank transfer (automatic),70.35,3454.6,No
5730,Male,0,Yes,No,27,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Bank transfer (automatic),19.55,520.55,No


In [7]:
# convert TotalCharges from categorical values to numerical values
df_customer["TotalCharges"] = pd.to_numeric(df_customer.TotalCharges,errors = 'coerce')

In [10]:
# check the unique values of each feature
def print_unique_col_values(df):
    for column in df:
        if df[column].dtype == 'object':
            print(f'{column}: {df[column].unique()}')

print_unique_col_values(df_customer)

gender: ['Female' 'Male']
Partner: ['Yes' 'No']
Dependents: ['No' 'Yes']
PhoneService: ['No' 'Yes']
MultipleLines: ['No phone service' 'No' 'Yes']
InternetService: ['DSL' 'Fiber optic' 'No']
OnlineSecurity: ['No' 'Yes' 'No internet service']
OnlineBackup: ['Yes' 'No' 'No internet service']
DeviceProtection: ['No' 'Yes' 'No internet service']
TechSupport: ['No' 'Yes' 'No internet service']
StreamingTV: ['No' 'Yes' 'No internet service']
StreamingMovies: ['No' 'Yes' 'No internet service']
Contract: ['Monthtomonth' 'Two year' 'One year']
PaperlessBilling: ['Yes' 'No']
PaymentMethod: ['Electronic check' 'Mailed check' 'Credit card (automatic)'
 'Bank transfer (automatic)']
Churn: ['No' 'Yes']


In [ ]:
# convert yes to 1, no to 0 for analysis
mapping_dict = {'Female': 0, 'Male': 1, 'No': 0, 'Yes': 1, 'No phone service': 0.5, 
                'DSL': 0.5, 'No internet service': 0, 'No internet': 0, 'No phone': 0}

cols_to_replace = ['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 
                   'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 
                   'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling', 'Churn']

df_customer[cols_to_replace] = df_customer[cols_to_replace].replace(mapping_dict)


In [6]:
# convert yes to 1, no to 0 for analysis

df_customer['gender'] = df_customer['gender'].apply(lambda x: 0 if x=='Female' else 1)
df_customer['Partner'] = df_customer['Partner'].apply(lambda x: 0 if x=='No' else 1)
df_customer['Dependents'] = df_customer['Dependents'].apply(lambda x: 0 if x=='No' else 1)
df_customer['PhoneService'] = df_customer['PhoneService'].apply(lambda x: 0 if x=='No' else 1)

df_customer['MultipleLines'] = df_customer['MultipleLines'].apply(lambda x: 0 if x=='No' else (0.5 if x=='No phone service' else 1))
df_customer['InternetService'] = df_customer['InternetService'].apply(lambda x: 0 if x=='No' else (0.5 if x=='DSL' else 1))
df_customer['OnlineSecurity'] = df_customer['OnlineSecurity'].apply(lambda x: 0 if x=='No' else (0.5 if x=='No internet service' else 1))
df_customer['OnlineBackup'] = df_customer['OnlineBackup'].apply(lambda x: 0 if x=='No' else (0.5 if x=='No internet service' else 1))
df_customer['DeviceProtection'] = df_customer['DeviceProtection'].apply(lambda x: 0 if x=='No' else (0.5 if x=='No internet service' else 1))
df_customer['TechSupport'] = df_customer['TechSupport'].apply(lambda x: 0 if x=='No' else (0.5 if x=='No internet service' else 1))
df_customer['StreamingTV'] = df_customer['StreamingTV'].apply(lambda x: 0 if x=='No' else (0.5 if x=='No internet service' else 1))
df_customer['StreamingMovies'] = df_customer['StreamingMovies'].apply(lambda x: 0 if x=='No' else (0.5 if x=='No internet service' else 1))


df_customer['PaperlessBilling'] = df_customer['PaperlessBilling'].apply(lambda x: 0 if x=='No' else 1)
df_customer['Churn'] = df_customer['Churn'].apply(lambda x: 0 if x=='No' else 1)

In [7]:
df_customer.sample(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
1430,4446-BZKHU,1,0,1,0,61,1,1.0,1.0,0.0,...,1.0,0.0,1.0,1.0,Monthtomonth,1,Electronic check,105.55,6281.45,1
3451,3296-SILRA,0,1,1,0,1,1,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,Monthtomonth,1,Mailed check,76.40,76.4,1
5056,7452-FOLON,1,0,0,1,39,1,1.0,0.0,0.5,...,0.5,0.5,0.5,0.5,One year,0,Bank transfer (automatic),25.45,958.45,0
6845,3544-FBCAS,0,0,1,1,72,1,1.0,0.5,1.0,...,1.0,0.0,1.0,1.0,Two year,1,Credit card (automatic),83.60,5959.3,0
6826,3719-TDVQB,0,1,1,0,54,1,0.0,0.0,0.5,...,0.5,0.5,0.5,0.5,Two year,1,Mailed check,18.95,1031.1,0


In [6]:
df_customer["TotalCharges"] = pd.to_numeric(df_customer.TotalCharges,errors = 'coerce')

In [7]:
df_customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   int64  
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   int64  
 4   Dependents        7043 non-null   int64  
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   int64  
 7   MultipleLines     7043 non-null   float64
 8   InternetService   7043 non-null   float64
 9   OnlineSecurity    7043 non-null   float64
 10  OnlineBackup      7043 non-null   float64
 11  DeviceProtection  7043 non-null   float64
 12  TechSupport       7043 non-null   float64
 13  StreamingTV       7043 non-null   float64
 14  StreamingMovies   7043 non-null   float64
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   int64  


In [8]:
df_customer.dropna()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,0,0,1,0,1,0,0.5,0.5,0.0,...,0.0,0.0,0.0,0.0,Monthtomonth,1,Electronic check,29.85,29.85,0
1,3668-QPYBK,1,0,0,0,2,1,0.0,0.5,1.0,...,0.0,0.0,0.0,0.0,Monthtomonth,1,Mailed check,53.85,108.15,1
2,9237-HQITU,0,0,0,0,2,1,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,Monthtomonth,1,Electronic check,70.70,151.65,1
3,9305-CDSKC,0,0,0,0,8,1,1.0,1.0,0.0,...,1.0,0.0,1.0,1.0,Monthtomonth,1,Electronic check,99.65,820.50,1
4,1452-KIOVK,1,0,0,1,22,1,1.0,1.0,0.0,...,0.0,0.0,1.0,0.0,Monthtomonth,1,Credit card (automatic),89.10,1949.40,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,4807-IZYOZ,0,0,0,0,51,1,0.0,0.0,0.5,...,0.5,0.5,0.5,0.5,Two year,0,Bank transfer (automatic),20.65,1020.75,0
7039,9710-NJERN,0,0,0,0,39,1,0.0,0.0,0.5,...,0.5,0.5,0.5,0.5,Two year,0,Mailed check,20.15,826.00,0
7040,9281-CEDRU,0,0,1,0,68,1,0.0,0.5,0.0,...,0.0,1.0,1.0,0.0,Two year,0,Bank transfer (automatic),64.10,4326.25,0
7041,2569-WGERO,0,0,0,0,72,1,0.0,0.0,0.5,...,0.5,0.5,0.5,0.5,Two year,1,Bank transfer (automatic),21.15,1419.40,0


In [17]:
df_customer_clean = pd.get_dummies(df_customer,columns=['Contract','PaymentMethod'])
df_customer_clean

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,MonthlyCharges,TotalCharges,Churn,Contract_Monthtomonth,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,7590-VHVEG,0,0,1,0,1,0,0.5,0.5,0.0,...,29.85,29.85,0,1,0,0,0,0,1,0
1,3668-QPYBK,1,0,0,0,2,1,0.0,0.5,1.0,...,53.85,108.15,1,1,0,0,0,0,0,1
2,9237-HQITU,0,0,0,0,2,1,0.0,1.0,0.0,...,70.70,151.65,1,1,0,0,0,0,1,0
3,9305-CDSKC,0,0,0,0,8,1,1.0,1.0,0.0,...,99.65,820.50,1,1,0,0,0,0,1,0
4,1452-KIOVK,1,0,0,1,22,1,1.0,1.0,0.0,...,89.10,1949.40,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,4807-IZYOZ,0,0,0,0,51,1,0.0,0.0,0.5,...,20.65,1020.75,0,0,0,1,1,0,0,0
7039,9710-NJERN,0,0,0,0,39,1,0.0,0.0,0.5,...,20.15,826.00,0,0,0,1,0,0,0,1
7040,9281-CEDRU,0,0,1,0,68,1,0.0,0.5,0.0,...,64.10,4326.25,0,0,0,1,1,0,0,0
7041,2569-WGERO,0,0,0,0,72,1,0.0,0.0,0.5,...,21.15,1419.40,0,0,0,1,1,0,0,0


In [18]:
df_customer_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 26 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   customerID                               7043 non-null   object 
 1   gender                                   7043 non-null   int64  
 2   SeniorCitizen                            7043 non-null   int64  
 3   Partner                                  7043 non-null   int64  
 4   Dependents                               7043 non-null   int64  
 5   tenure                                   7043 non-null   int64  
 6   PhoneService                             7043 non-null   int64  
 7   MultipleLines                            7043 non-null   float64
 8   InternetService                          7043 non-null   float64
 9   OnlineSecurity                           7043 non-null   float64
 10  OnlineBackup                             7043 no

2. Modelling

In [19]:
X = df_customer_clean.drop(columns = ['customerID','Churn']).copy().astype(float)
Y = np.ravel(df_customer_clean[['Churn']])

In [20]:
X

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,...,PaperlessBilling,MonthlyCharges,TotalCharges,Contract_Monthtomonth,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0.0,0.0,1.0,0.0,1.0,0.0,0.5,0.5,0.0,1.0,...,1.0,29.85,29.85,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0,2.0,1.0,0.0,0.5,1.0,1.0,...,1.0,53.85,108.15,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,2.0,1.0,0.0,1.0,0.0,0.0,...,1.0,70.70,151.65,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,8.0,1.0,1.0,1.0,0.0,0.0,...,1.0,99.65,820.50,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1.0,0.0,0.0,1.0,22.0,1.0,1.0,1.0,0.0,1.0,...,1.0,89.10,1949.40,1.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,0.0,0.0,0.0,0.0,51.0,1.0,0.0,0.0,0.5,0.5,...,0.0,20.65,1020.75,0.0,0.0,1.0,1.0,0.0,0.0,0.0
7039,0.0,0.0,0.0,0.0,39.0,1.0,0.0,0.0,0.5,0.5,...,0.0,20.15,826.00,0.0,0.0,1.0,0.0,0.0,0.0,1.0
7040,0.0,0.0,1.0,0.0,68.0,1.0,0.0,0.5,0.0,1.0,...,0.0,64.10,4326.25,0.0,0.0,1.0,1.0,0.0,0.0,0.0
7041,0.0,0.0,0.0,0.0,72.0,1.0,0.0,0.0,0.5,0.5,...,1.0,21.15,1419.40,0.0,0.0,1.0,1.0,0.0,0.0,0.0


In [21]:
Y

array([0, 1, 1, ..., 0, 0, 0], dtype=int64)

In [22]:
# training and test sets
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X, Y, test_size=0.20, random_state=23)

In [23]:
X_train

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,...,PaperlessBilling,MonthlyCharges,TotalCharges,Contract_Monthtomonth,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
6797,0.0,0.0,0.0,1.0,43.0,1.0,1.0,0.0,0.5,0.5,...,1.0,25.10,1070.15,0.0,0.0,1.0,0.0,0.0,1.0,0.0
6847,1.0,0.0,0.0,0.0,60.0,1.0,1.0,1.0,0.0,0.0,...,1.0,89.55,5231.20,0.0,0.0,1.0,0.0,0.0,1.0,0.0
5699,1.0,0.0,0.0,1.0,41.0,1.0,0.0,1.0,1.0,1.0,...,1.0,111.15,4507.15,0.0,0.0,1.0,0.0,0.0,1.0,0.0
6221,0.0,0.0,0.0,0.0,49.0,1.0,0.0,0.5,0.0,0.0,...,1.0,74.45,3721.90,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1048,0.0,0.0,1.0,0.0,46.0,0.0,0.5,0.5,0.0,0.0,...,1.0,43.95,2007.85,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2998,0.0,0.0,0.0,0.0,15.0,1.0,0.0,1.0,0.0,0.0,...,1.0,94.00,1505.45,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1993,0.0,0.0,0.0,0.0,12.0,1.0,1.0,1.0,0.0,0.0,...,1.0,100.15,1164.30,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1064,0.0,1.0,1.0,1.0,22.0,1.0,1.0,1.0,0.0,0.0,...,0.0,85.35,1961.60,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4838,0.0,0.0,0.0,0.0,23.0,1.0,0.0,0.0,0.5,0.5,...,1.0,19.65,478.10,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [24]:
# one tempt, check if gender, seniorcitizen, partner, contract have relationship with churn
X_gender_member = X_train[['gender','SeniorCitizen','Partner','Contract_Monthtomonth','Contract_One year','Contract_Two year']]
X_gender_membe_test = X_test[['gender','SeniorCitizen','Partner','Contract_Monthtomonth','Contract_One year','Contract_Two year']]

In [25]:
log_Reg = LogisticRegression()

In [27]:
log_Reg.fit(X_gender_member,Y_train)

LogisticRegression()